# The scripts of the bioinformatic workflow to identify putative PET-degrading enzymes

## Step 1
First, we **extract all sequences** that contain a contiguous sequence of SMGGG. Second, we extract all sequences that contain a D between 25 and 35 characters after SMGGG. Third, we extract all sequences that contain an H between 68 and 78 characters after SMGGG.  
Download the **Alphafold_sequences.fasta** on the website: https://alphafold.ebi.ac.uk/download

In [2]:
! grep -B 1 SMGGG path/to/Alphafold_sequences.fasta | grep -E -B 1 "SMGGG.{25,35}D" | grep -E -B 1 "SMGGG.{68,78}H" | grep -v ^- > SMGGG_D_H.fasta

## Step 2
Extract the **description IDs** of all sequences in the SMGGG_D_H.fasta file, which will be used to download the corresponding PDB files from the AlphaFold database.

In [ ]:
! grep \> path/to/SMGGG_D_H.fasta| awk -F'[- ]' '/AF-/,/-F1/{print $2}' > AFDB_download_list.txt

## Step 3
Download the corresponding PDB files from the AlphaFold database according to the **AFDB_download_list.txt** file.

In [ ]:
! cat AFDB_download_list.txt | xargs -I {} -n 1 -P 4 wget -P path/to/AF_PDB(folder to storage the pdb files)/ "https://alphafold.ebi.ac.uk/files/AF-{}-F1-model_v4.pdb"

## Step 4
For each sequence in the FASTA file, find the possible permutations of the positions of S, D, and H in the SMGGG sequence, which will be used to calculate the distance between S, D, and H in the PDB file.

In [ ]:
import os
import math

# Difine the function to parse the fasta file
def parse_fasta(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    protein_data = []
    current_protein = {"name": "", "sequence": ""}
    
    for line in lines:
        line = line.strip()
        if line.startswith(">"):
            if current_protein["name"] != "":
                protein_data.append(current_protein.copy())
            current_protein["name"] = line.split(" ", 1)[0][1:]  # Extract the part before the first space
            current_protein["sequence"] = ""
        else:
            current_protein["sequence"] += line

    protein_data.append(current_protein)  # Add the last protein
    
    return protein_data

# Define the function to find the position of the target substring
def find_s_position(sequence, target_substring):
    positions = [i + 1 for i in range(len(sequence)) if sequence[i:i + len(target_substring)] == target_substring]
    return positions

# Define the function to find the position of the target char in the interval
def find_char_positions_in_interval(sequence, start, end, target_char):
    positions = [i + 1 for i in range(start - 1, min(end, len(sequence))) if sequence[i] == target_char]
    return positions

# Define the function to get the coordinates and reliability of the target position
def get_ca_coordinates_and_reliability(pdb_path, protein_name, i):
    pdb_file_path = os.path.join(pdb_path, f"{protein_name}-model_v4.pdb")

    try:
        with open(pdb_file_path, "r") as f:
            ca_lines = [line.split() for line in f if line.startswith("ATOM") and " CA " in line]

            if 1 <= i <= len(ca_lines):
                line = ca_lines[i - 1]
                x = float(line[6])
                y = float(line[7])
                z = float(line[8])
                reliability = float(line[10])

                return {"coordinates": [x, y, z], "reliability": reliability}
            else:
                print(f"Error: Index {i} is out of range.")
                return None

    except (ValueError, IndexError) as e:
        print(f"Error: {e}")
        return None

# Define the function to calculate the distance between two points
def euclidean_distance(point1, point2):
    if len(point1) != len(point2):
        raise ValueError("Points must have the same number of dimensions")

    squared_distances = [(p1 - p2)**2 for p1, p2 in zip(point1, point2)]
    sum_of_squares = sum(squared_distances)
    distance = math.sqrt(sum_of_squares)
    
    return distance

# Define the main function
def main():
    fasta_file_path = "path/to/SMGGG_D_H.fasta"  # Replace with your actual file path
    protein_data = parse_fasta(fasta_file_path)
    pdb_path = "/path/to/AF_PDB(folder to storage the pdb files)"
    
    for protein in protein_data:
        protein_name = protein["name"].replace("AFDB:", "")

        # Find position of 'S' in SMGGG
        s_positions = find_s_position(protein["sequence"], "SMGGG")

        # Find positions of 'D'
        d_positions = find_char_positions_in_interval(protein["sequence"], s_positions[0] + len("SMGGG") + 25, s_positions[0] + len("SMGGG") + 35, 'D')

        # Find positions of 'H'
        h_positions = find_char_positions_in_interval(protein["sequence"], s_positions[0] + len("SMGGG") + 68, s_positions[0] + len("SMGGG") + 78, 'H')

        # Find the distance between S and D, S and H, D and H
        with open(os.path.join(pdb_path, protein_name + "-model_v4.pdb"), "r") as f:
            for i in range(len(s_positions)):
                for j in range(len(d_positions)):
                    for k in range(len(h_positions)):
                        # Find the positions of S, D, H
                        result_positions = []
                        result_positions.append(s_positions[i])
                        result_positions.append(d_positions[j])
                        result_positions.append(h_positions[k])
                        
                        # Find the coordinates and reliability of S, D, H
                        Ca_coordinates = [get_ca_coordinates_and_reliability(pdb_path, protein_name, i)["coordinates"] for i in result_positions]
                        Ca_reliability = [get_ca_coordinates_and_reliability(pdb_path, protein_name, i)["reliability"] for i in result_positions]
                        
                        # Calculate the distance between S, D, H, and determine whether the distance meets the requirements of reliability >= 60.
                        if min(Ca_reliability) >= 60:
                            S_D_distance = euclidean_distance(Ca_coordinates[0], Ca_coordinates[1])
                            S_H_distance = euclidean_distance(Ca_coordinates[0], Ca_coordinates[2])
                            D_H_distance = euclidean_distance(Ca_coordinates[1], Ca_coordinates[2])
                            if 20 <= S_D_distance <= 22 and 25 <= S_H_distance <= 27 and 37 <= D_H_distance <= 39:
                                print("Protein Name:", protein_name)
                                print("Result Positions:", result_positions)
                                print("S_D_distance:", S_D_distance)
                                print("S_H_distance:", S_H_distance)
                                print("D_H_distance:", D_H_distance)
                            

if __name__ == "__main__":
    main()